# Imports

In [48]:
import pandas as pd
import numpy as np
#data visualization
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam
from keras.constraints import maxnorm

pd.options.display.precision = 10

# Umgebungsvariablen

In [40]:
#feature generated 24.03.2019
# path data Domenico
earthquake_daten = '''C:/Users/taacodo4/GITRepos/CAS_PML_Earthquake_pred/data/'''
# path data Stefan
#earthquake_daten = '''C:/studium/studium/CAS_PML/Projekt_Arbeit/earthquake/Daten/earthquake_data/'''
feature_62900_94 = 'Features_62900-94.csv'
feature_41934_94 = 'Features_41934_94.csv'
Features_4194_94 = 'Features_4194_94.csv'

# Feature laden

In [41]:
#train_data = pd.read_csv(earthquake_daten+feature_41934_94)
train_data = pd.read_csv(earthquake_daten+Features_4194_94)

In [42]:
feature_data = train_data.iloc[:,1:len(train_data.columns)-1] # first column is an index column
time_to_failure = np.array(train_data.iloc[:,-1])

In [43]:
input_variables=feature_data
output_variable=time_to_failure

In [44]:
feature_data.head()

,mean,std,kurt,skew,min,max,1%q,5%q,25%q,75%q,...,spec_bw_kurt,spec_bw_skew,spec_bw_min,spec_bw_max,spec_bw_1%q,spec_bw_5%q,spec_bw_25%q,spec_bw_75%q,spec_bw_95%q,spec_bw_99%q
0,4.8841133333,5.1011061306,33.6624812935,-0.0240611666,-98,104,-8.0,-2.0,3.0,7.0,...,12.7760446198,-2.9176274163,381349.1865900679,593848.0949585186,464802.9437660335,513229.8197951605,557429.6937546992,578910.9580815419,586597.8954874629,589247.3423812583
1,4.7257666667,6.5888237819,98.7585171787,0.3905605044,-154,181,-11.0,-2.0,2.0,7.0,...,12.4473098885,-3.1420260522,339068.4229139190,593047.2583837381,373488.0462558223,487974.0669297441,553402.7519102279,578052.3179053498,586268.5112265539,589480.9659446193
2,4.9063933333,6.9673970335,33.5552114069,0.2173905614,-106,140,-15.0,-3.0,2.0,7.0,...,4.9488288962,-2.1608426205,344167.5096713643,592738.7943923724,397152.4604941708,465235.6930021432,545439.4027226671,577177.5071075818,584977.6504044535,588695.0354595457
3,4.9022400000,6.9223051872,116.5481716876,0.7572775364,-199,197,-12.0,-2.0,2.0,7.0,...,11.6064831292,-3.0859627131,317909.0883782429,594108.2442517320,395493.9927363199,481201.4155927489,554046.7131943784,578726.4379399519,586329.5304609182,592069.8251146671
4,4.9087200000,7.3011101898,52.9779048344,0.0645310693,-126,145,-15.0,-2.0,2.0,7.0,...,9.9730546565,-2.9666542555,327985.3002366525,592276.8748307980,363126.2614921932,469766.1652834315,554713.0297514369,579583.0722189071,585261.5016828560,590190.4467753589


# Base model

## Training / Testdaten vorbereiten

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    feature_data,
    time_to_failure,
    random_state=0,
    test_size=0.25)

In [6]:
scaler = StandardScaler().fit(X_train)
X_train_scaled=scaler.transform(X_train)

C:\Users\chianti\.conda\envs\TensorFlow-GPU-Keras\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\chianti\.conda\envs\TensorFlow-GPU-Keras\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


## model compilation

In [ ]:
model = Sequential()
model.add(Dense(30, activation='relu', input_dim=94))
model.add(Dense(40, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mae',optimizer='sgd',metrics=['mae', 'acc'],)
#model.compile(loss='mae', optimizer='adam')

model.fit(X_train_scaled, y_train, epochs=100, batch_size=10)

In [21]:
X_test_scaled=scaler.transform(X_test)

C:\Users\taacodo4\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 30)                2850      
_________________________________________________________________
dense_18 (Dense)             (None, 40)                1240      
_________________________________________________________________
dense_19 (Dense)             (None, 30)                1230      
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 31        
Total params: 5,351
Trainable params: 5,351
Non-trainable params: 0
_________________________________________________________________


In [23]:
prediction=model.predict(X_test_scaled)
#Print model report:
print("\nModel Report")
print(model.summary)
print("MAE train data: %.4g" % mean_absolute_error(y_test, prediction))


Model Report
<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x00000123187E60F0>>
MAE train data: 1.774


In [ ]:
prediction=model.predict(X_test_scaled)
#Print model report:
print("\nModel Report")
print("Settings: PCA 20, 30/40/30, 100 epochs, 10 batch size, optimizer sgd, loss function mae")
print(model.summary)
print("MAE train data: %.4g" % mean_absolute_error(y_test, prediction))

# Tuning hyperparameter

Grid search FFN:
https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

Design of NN:
https://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,2&seed=0.19677&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false

Loss function: 
https://isaacchanghau.github.io/post/loss_functions/

## Concept
The neuronal network shall be optimized on following hyperparameter:
* optimization algorithms
* batch size and training epochs
* learning rate and momentum
* network weight initialization
* activation function
* dropout regularization
* number of neurons in the hidden layer

For this purpose the smaller data set of 4k shall be used.

## Optimization algorithm
https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/

Regression Problem<br>
A problem where you predict a real-value quantity.

Output Layer Configuration: One node with a linear activation unit.<br>
Loss Function: Mean Squared Error (MSE).

In [55]:
model = Sequential()
model.add(Dense(30, activation='relu', input_dim=94))
model.add(Dense(40, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mae',optimizer='sgd',metrics=['mae', 'acc'],)
#model.compile(loss='mae', optimizer='adam')

model.fit(input_variables, output_variable, epochs=10, batch_size=1)

Epoch 1/10
4194/4194 [==============================] - 3s 697us/step - loss: 1810468526596012032.0000 - mean_absolute_error: 1810468526596012032.0000 - acc: 0.0000e+00
Epoch 2/10
4194/4194 [==============================] - 2s 570us/step - loss: 3.0426 - mean_absolute_error: 3.0426 - acc: 0.0000e+00
Epoch 3/10
4194/4194 [==============================] - 2s 549us/step - loss: 3.0422 - mean_absolute_error: 3.0422 - acc: 0.0000e+00
Epoch 4/10
4194/4194 [==============================] - 2s 571us/step - loss: 3.0398 - mean_absolute_error: 3.0398 - acc: 0.0000e+00
Epoch 5/10
4194/4194 [==============================] - 3s 617us/step - loss: 3.0420 - mean_absolute_error: 3.0420 - acc: 0.0000e+00
Epoch 6/10
4194/4194 [==============================] - 2s 562us/step - loss: 3.0423 - mean_absolute_error: 3.0423 - acc: 0.0000e+00
Epoch 7/10
4194/4194 [==============================] - 2s 572us/step - loss: 3.0418 - mean_absolute_error: 3.0418 - acc: 0.0000e+00
Epoch 8/10
4194/4194 [===========

In [56]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='Adam'):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=94, activation='relu'))
    model.add(Dense(1, activation='linear'))
    # Compile model
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

# create model
pipe = Pipeline([('preprocessing', StandardScaler()), ('Feed_Forward', KerasRegressor(build_fn=create_model, epochs=1, batch_size=1, verbose=0))])

# define the grid search parameters
param_grid = [{'Feed_Forward__optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']}]

grid = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=5,scoring='neg_mean_absolute_error', n_jobs=-1)
grid_result = grid.fit(input_variables, output_variable)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

## Batch size and number of Epoche

In [17]:
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=94, activation='relu'))
    model.add(Dense(1, activation='linear'))
    # Compile model
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5,scoring='neg_mean_absolute_error', n_jobs=-1)
grid_result = grid.fit(input_variables, output_variable)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.000477 using {'batch_size': 10, 'epochs': 100}
0.000238 (0.000477) with: {'batch_size': 10, 'epochs': 10}
0.000238 (0.000477) with: {'batch_size': 10, 'epochs': 50}
0.000477 (0.000584) with: {'batch_size': 10, 'epochs': 100}
0.000238 (0.000477) with: {'batch_size': 20, 'epochs': 10}
0.000000 (0.000000) with: {'batch_size': 20, 'epochs': 50}
0.000000 (0.000000) with: {'batch_size': 20, 'epochs': 100}
0.000000 (0.000000) with: {'batch_size': 40, 'epochs': 10}
0.000477 (0.000584) with: {'batch_size': 40, 'epochs': 50}
0.000238 (0.000477) with: {'batch_size': 40, 'epochs': 100}
0.000000 (0.000000) with: {'batch_size': 60, 'epochs': 10}
0.000000 (0.000000) with: {'batch_size': 60, 'epochs': 50}
0.000238 (0.000477) with: {'batch_size': 60, 'epochs': 100}
0.000238 (0.000477) with: {'batch_size': 80, 'epochs': 10}
0.000238 (0.000477) with: {'batch_size': 80, 'epochs': 50}
0.000238 (0.000477) with: {'batch_size': 80, 'epochs': 100}
0.000477 (0.000584) with: {'batch_size': 100, 'epochs':

## Learning Rate and Momentum

In [28]:
def create_model(learn_rate=0.01):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=94, activation='relu'))
    model.add(Dense(1, activation='lin'))
    # Compile model
    optimizer = Adam(lr=learn_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

In [29]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5, n_jobs=-1)
grid_result = grid.fit(input_variables, output_variable)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.000477 using {'learn_rate': 0.001}
0.000477 (0.000584) with: {'learn_rate': 0.001}
0.000238 (0.000477) with: {'learn_rate': 0.01}
0.000000 (0.000000) with: {'learn_rate': 0.1}
0.000000 (0.000000) with: {'learn_rate': 0.2}
0.000238 (0.000477) with: {'learn_rate': 0.3}


## Network Weight Initialization

In [31]:
def create_model(init_mode='uniform'):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=94, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5, n_jobs=-1)
grid_result = grid.fit(input_variables, output_variable)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.000477 using {'init_mode': 'he_normal'}
0.000000 (0.000000) with: {'init_mode': 'uniform'}
0.000000 (0.000000) with: {'init_mode': 'lecun_uniform'}
0.000238 (0.000477) with: {'init_mode': 'normal'}
0.000238 (0.000477) with: {'init_mode': 'zero'}
0.000238 (0.000477) with: {'init_mode': 'glorot_normal'}
0.000238 (0.000477) with: {'init_mode': 'glorot_uniform'}
0.000477 (0.000584) with: {'init_mode': 'he_normal'}
0.000000 (0.000000) with: {'init_mode': 'he_uniform'}


## Neuron Activation Function

In [33]:
# Function to create model, required for KerasClassifier
def create_model(activation='relu'):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=94, kernel_initializer='he_normal', activation=activation))
    model.add(Dense(1, kernel_initializer='he_normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)
grid_result = grid.fit(input_variables, output_variable)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.000477 using {'activation': 'linear'}
0.000238 (0.000477) with: {'activation': 'softmax'}
0.000000 (0.000000) with: {'activation': 'softplus'}
0.000238 (0.000477) with: {'activation': 'softsign'}
0.000238 (0.000477) with: {'activation': 'relu'}
0.000238 (0.000477) with: {'activation': 'tanh'}
0.000238 (0.000477) with: {'activation': 'sigmoid'}
0.000238 (0.000477) with: {'activation': 'hard_sigmoid'}
0.000477 (0.000584) with: {'activation': 'linear'}


## Dropout Regularization

In [35]:
# Function to create model, required for KerasClassifier
def create_model(dropout_rate=0.0, weight_constraint=0):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=94, kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.000238 using {'activation': 'softmax'}
0.000238 (0.000477) with: {'activation': 'softmax'}
0.000000 (0.000000) with: {'activation': 'softplus'}
0.000238 (0.000477) with: {'activation': 'softsign'}
0.000000 (0.000000) with: {'activation': 'relu'}
0.000238 (0.000477) with: {'activation': 'tanh'}
0.000238 (0.000477) with: {'activation': 'sigmoid'}
0.000000 (0.000000) with: {'activation': 'hard_sigmoid'}
0.000000 (0.000000) with: {'activation': 'linear'}


## Number of Neurons in the Hidden Layer

In [ ]:
# Function to create model, required for KerasClassifier
def create_model(neurons=1):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=94, kernel_initializer='he_normal', activation='linear', kernel_constraint=maxnorm(4)))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='he_normal', activation='linear'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

#create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(input_variables, output_variable)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))